Here, we show how to apply FlowSig to an scRNA-seq dataset of wildtype
and stimulated human pancreatic islets, as originally studied in [Burkhardt et al. (2021)](https://www.nature.com/articles/s41587-020-00803-5).

In [ ]:
import scanpy as sc
import pandas as pd
import flowsig as fs

Load the data and set the observation key that we will use to split the data into control (observational) and perturbed (interventional data). 

In [ ]:
adata = sc.read('data/burkhardt21_merged.h5ad')
condition_key = 'Condition'

Load the cell-cell communication output from [CellChat](https://www.nature.com/articles/s41467-021-21246-9), which has been run for each condition.

In [ ]:
cellchat_Ctrl = pd.read('data/burkhardt21_leiden_communications_Ctrl.csv')
cellchat_IFNg = pd.read('data/burkhardt21_leiden_communications_IFNg.csv')

cellchat_output_key = 'cellchat_output'
# Make sure your keys for these align with their condition labels
adata.uns[cellchat_output_key] = {'Ctrl': cellchat_Ctrl,
                                  'IFNg': cellchat_IFNg}

We construct 10 gene expression modules from the unnormalized gene expression counts using [pyLIGER](https://academic.oup.com/bioinformatics/article/38/10/2946/6561542), which uses iNMF to construct GEMs that account for shared and specific expression across the control and perturbed conditions.

In [ ]:
# We construct 10 gene expression modules using the raw cell count.
fs.pp.construct_gems_using_pyliger(adata,
                                n_gems = 10,
                                layer_key = 'counts',
                                condition_key = condition_key)

We first construct the potential cellular flows from the cellchat output, i.e., separate the inflows from the outflows.

In [ ]:
fs.pp.construct_flow_expressions(adata,
                                cellchat_output_key=cellchat_output_key,
                                model_organism = 'human',
                                spatial = False,
                                method = 'cellchat'
                                )

Then we subset for "differentially flowing" variables, using a Mann-Whitney U test on the inflow and outflow expressions, separately.

In [ ]:
fs.pp.determine_informative_variables(adata,  
                                    spatial = False,
                                    condition_key = 'Condition',
                                    control = 'Ctrl',
                                    qval_threshold = 0.05,
                                    logfc_threshold = 0.5)

Now we are ready to learn the network

In [ ]:
fs.tl.learn_intercellular_flows(adata,
                        condition_key = condition_key,
                        control = 'Ctrl', 
                        use_spatial = False,
                        n_jobs = 1,
                        n_bootstraps = 10)

Now we do post-learning validation to reorient the network and remove low-quality edges.


In [ ]:
# This part is key for reducing false positives
fs.tl.apply_biological_flow(adata,
                        flowsig_network_key = 'flowsig_network',
                        adjacency_key = 'adjacency',
                        validated_key = 'adjacency_validated')

edge_threshold = 0.7

fs.tl.filter_low_confidence_edges(adata,
                                edge_threshold = edge_threshold,
                                flowsig_network_key = 'flowsig_network',
                                adjacency_key = 'adjacency',
                                filtered_key = 'adjacency_filtered')

adata.write('data/burkhardt21_merged.h5ad', compression='gzip')